In [20]:
from pathlib import Path
import sys
import yaml
import time

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

DIR = Path('..')
sys.path.append(str(Path('..').resolve()))
from SCanalyzer import SCanalyzer
from SCanalyzer.busSim import BusSim, Config
from SCanalyzer.busSim.manager import managerFactory
from SCanalyzer.gtfs_edit import edit_double

%matplotlib inline

In [2]:
def get_yelp_api_key():
    with open("../config.yml", 'r') as yml:
        cfg = yaml.safe_load(yml)
        return cfg["yelp"]["api_key"]

def plot_background():
    background_path = "../data/plot/background/"
    city = gpd.read_file(background_path + "madison-meter-shp")
    lakes = gpd.read_file(background_path + "water-meter-shp")
    street = gpd.read_file(background_path + "street-meter-shp")
    # city = gpd.read_file(background_path + "madison-shp")
    # lakes = gpd.read_file(background_path + "water-shp")
    # street = gpd.read_file(background_path + "street-shp")

    ax = city.plot(color="lightgray", alpha=.2, figsize=(12, 12), zorder=2)
    lakes.plot(color="lightblue", ax=ax, zorder=1, alpha=0.8)
    street.plot(color="darkgray", alpha = .5, ax=ax, zorder=3)
    return ax

# route remove study

In [4]:
api_key = 'ZsHZFGtKEZeOOXhTne98eErmfd6BfNTm9GqO2S6inSnWzDwtgC2sEauXcB-8zUna_lXZEal4jsW_St6O0OQOcuNvifrr6uqNYmjFXW-FyVKvaMyczbihWELI80tjYHYx'
gtfo = SCanalyzer("../data/mmt_gtfs.zip")
census_gdf = gtfo.load_census()
services_gdf = gtfo.load_yelp(api_key=api_key)

In [6]:
# routes = [ 2,  4,  5,  6,  7,  8, 10, 11, 12, 13, 15, 16, 17, 18, 20, 21, 22,
#        23, 26, 27, 30, 31, 32, 33, 36, 39, 40, 44, 48, 49, 50, 51, 52, 55,
#        67, 70, 72, 73, 75, 59, 63, 68, 78, 80, 84, 81, 82]

routes = [80]
for route in routes:
    print(route)
    gtfo.set_batch_label(f"{route}-rm")
    config = Config(day="monday", elapse_time="00:30:00", interval="02:00:00", max_walking_min=10, route_remove=[route])
    config.set_starts(centroids=census_gdf)

    perf_df = pd.DataFrame(
            columns=["geometry", "start_time", "search_time", "add_service_time", "add_census_time"])
    result_gdf = gtfo.search(config, perf_df)
    gtfo.add_service_metrics(result_gdf, services_gdf, perf_df)
    gtfo.add_demographic_metrics(result_gdf, census_gdf, perf_df)
    result_gdf.to_csv(f"./out/result/result{route}.csv", index=False)
    perf_df.to_csv(f"./out/perf/perf{route}.csv", index=False)

80
Empty DataFrame
Columns: [geometry, start_time, search_time, add_service_time, add_census_time]
Index: []


  0%|                                                                                          | 0/3336 [00:00<?, ?it/s]/home/charles/.local/lib/python3.8/site-packages/pandas/core/internals/managers.py:327: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  applied = getattr(b, f)(**kwargs)
100%|██████████████████████████████████████████████████████████████████████████████| 3336/3336 [00:33<00:00, 100.03it/s]


In [7]:
result_gdf

,geometry,start_time,map_identifier,banks,clinics,dentists,hospitals,supermarket,STATE,COUNTY,...,B02001_005E,B02001_006E,B25044_001E,Tot Pop,% White alone,% Black or African American alone,% American Indian and Alaska Native alone,% Asian alone alone,% Native Hawaiian and Other Pacific Islander alone,cars per capita
0,POLYGON ((-89.68980799994183 43.11966599982823...,0:0:00,search-result-0-0!0,0.0,0.0,0.0,0.0,0.0,55.0,25.0,...,40.0,0.0,1045.0,2730.0,0.940659,0.003297,0.000000,0.014652,0.000000,0.382784
1,POLYGON ((-89.60231500001349 42.98794000007241...,0:0:00,search-result-0-0!1,0.0,0.0,0.0,0.0,0.0,55.0,25.0,...,81.0,0.0,702.0,2037.0,0.929308,0.000000,0.004909,0.039764,0.000000,0.344624
2,"POLYGON ((-89.5330390004105 43.03408800022495,...",0:0:00,search-result-0-0!2,0.0,0.0,0.0,0.0,0.0,55.0,25.0,...,438.0,0.0,1788.0,3385.0,0.823634,0.009453,0.000000,0.129394,0.000000,0.528213
3,POLYGON ((-89.53338500001689 43.02483299980272...,0:0:00,search-result-0-0!3,0.0,0.0,0.0,0.0,0.0,55.0,25.0,...,129.0,0.0,572.0,1408.0,0.897017,0.000000,0.000000,0.091619,0.000000,0.406250
4,POLYGON ((-89.51780899991446 43.04970000007612...,0:0:00,search-result-0-0!4,0.0,0.0,0.0,0.0,0.0,55.0,25.0,...,380.0,0.0,1075.0,2050.0,0.684390,0.064878,0.000000,0.185366,0.000000,0.524390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3331,"POLYGON ((-89.2651719998744 43.16857699986124,...",22:0:00,search-result-0-1320!273,0.0,0.0,0.0,0.0,0.0,55.0,25.0,...,164.0,4.0,995.0,2205.0,0.634467,0.230839,0.000000,0.074376,0.001814,0.451247
3332,"POLYGON ((-89.2262600001057 43.18746699968422,...",22:0:00,search-result-0-1320!274,0.0,0.0,0.0,0.0,0.0,55.0,25.0,...,30.0,0.0,423.0,1097.0,0.696445,0.236098,0.000000,0.027347,0.000000,0.385597
3333,"POLYGON ((-89.2470499999836 43.02714299999305,...",22:0:00,search-result-0-1320!275,0.0,0.0,0.0,0.0,0.0,55.0,25.0,...,0.0,5.0,242.0,450.0,0.988889,0.000000,0.000000,0.000000,0.011111,0.537778
3334,POLYGON ((-89.21130400014435 43.19163899984297...,22:0:00,search-result-0-1320!276,0.0,0.0,0.0,0.0,0.0,55.0,25.0,...,0.0,0.0,391.0,922.0,0.955531,0.030369,0.000000,0.000000,0.000000,0.424078


In [9]:
perf_df

,geometry,start_time,search_time,add_service_time,add_census_time
0,POINT (-89.65749071743502 43.12558522013633),0:0:00,0.005405,0.003044,0.007783
1,POINT (-89.58410468038977 42.9977144585),0:0:00,0.005658,0.003008,0.00106
2,POINT (-89.52480673380735 43.0440282770501),0:0:00,0.005282,0.003081,0.000483
3,POINT (-89.52461125108374 43.02680593102333),0:0:00,0.005286,0.003001,0.000545
4,POINT (-89.51016191651111 43.0479695906991),0:0:00,0.005009,0.003015,0.000588
...,...,...,...,...,...
3331,POINT (-89.2543335991909 43.17941696996946),22:0:00,0.007677,0.003035,0.000179
3332,POINT (-89.22025804318837 43.18544080303251),22:0:00,0.007228,0.003003,0.000194
3333,POINT (-89.23124966766352 43.05574718320674),22:0:00,0.007323,0.003129,0.000179
3334,POINT (-89.19161761144784 43.21318565393248),22:0:00,0.007422,0.003063,0.000176


# route doubling study

In [ ]:
print(perf_df)

In [ ]:
sc = SCanalyzer("../data/mmt_gtfs.zip")
census_gdf = sc.load_census()
services_gdf = sc.load_yelp(api_key=get_yelp_api_key())
config = Config(day="monday", elapse_time="00:30:00", interval="10:10:00", max_walking_min=10)
config.set_starts(centroids=census_gdf)

routes = [ 2,  4,  5,  6,  7,  8, 10, 11, 12, 13, 15, 16, 17, 18, 20, 21, 22,
       23, 26, 27, 30, 31, 32, 33, 36, 39, 40, 44, 48, 49, 50, 51, 52, 55,
       67, 70, 72, 73, 75, 59, 63, 68, 78, 80, 84, 81, 82]
routes = [80]
for route in routes:
    print(route)
    sc.gtfs_edit(edit_double, route)
    sc.set_batch_label(f"{route}-double")
    perf_df = pd.DataFrame(
            columns=["geometry", "start_time", "search_time", "add_service_time", "add_census_time"])
    result_gdf = sc.search(config, perf_df)
    sc.add_service_metrics(result_gdf, services_gdf, perf_df)
    sc.add_demographic_metrics(result_gdf, census_gdf, perf_df)
    result_gdf.to_csv(f"./out/result/result{route}-double.csv", index=False)
    perf_df.to_csv(f"./out/perf/perf{route}-double.csv", index=False)
    break

In [ ]:
sc = SCanalyzer("../data/mmt_gtfs.zip")
census_gdf = sc.load_census()
config = Config(day="monday", elapse_time="00:30:00", interval="10:00:00", max_walking_min=10)
# config.set_starts(points=[(43.073691, -89.387407), (43.073691, -89.387407)])
config.set_starts(centroids=census_gdf)
t0 = time.time()
result_gdf = sc.search(config)
t1 = time.time()
print(t1-t0)

services_gdf = sc.load_yelp(api_key=get_yelp_api_key())
sc.add_service_metrics(result_gdf, services_gdf)
t2 = time.time()
print(t2-t1)

sc.add_demographic_metrics(result_gdf, census_gdf) # SLOW
t3 = time.time()
print(t3-t2)
result_gdf["banks"].sum() #7263.0

In [ ]:
sc.gtfs_edit(edit_double, 80)
t0 = time.time()
result_gdf = sc.search(config)
t1 = time.time()
print(t1-t0)

services_gdf = sc.load_yelp(api_key=get_yelp_api_key())
sc.add_service_metrics(result_gdf, services_gdf)
t2 = time.time()
print(t2-t1)

sc.add_demographic_metrics(result_gdf, census_gdf) # SLOW
t3 = time.time()
print(t3-t2)
result_gdf["banks"].sum() #7263.0

In [ ]:
result = pd.read_csv("./out/result/result80-double.csv")
result["banks"].sum()

# Visualize simulation

In [ ]:
gtfo = Gtfo("../data/mmt_gtfs.zip")
ax = plot_background()
gtfo.set_batch_label(f"80-rm")
example_gdf = gtfo.load_result_map(map_identifier="search-result-0-600!10")
example_gdf.plot(color="#ffbfba", ax=ax)

# AWS test

In [ ]:
from gtfo.busSim.manager import managerFactory
aws = managerFactory.create("aws", gtfs_path=None, city_path=None, out_path=None)

In [ ]:
aws.clean_up()

In [ ]:
import boto3
aws_lambda = boto3.client('lambda', region_name='ap-northeast-1')
with open("tmp.zip", 'rb') as f:
    response = aws_lambda.create_function(
        Code={
            'ZipFile': f.read()
        },
        Description='BusSim handler',
        FunctionName="test",
        Handler='lambda_function.lambda_handler',
        MemorySize=512,
        Publish=True,
        Role="arn:aws:iam::347664766527:role/s3rwRole",
        Runtime='python3.8',
        Timeout=900,
        TracingConfig={
            'Mode': 'Active',
        },
    )